# Load data

In [3]:
!pip install rasterio

zsh:1: command not found: pip


In [2]:
import geopandas as gpd
import pandas as pd

In [4]:
subbasin_gdf = gpd.read_file('../data/raw/hydrography/gsl_catchment.shp')
gage_df = pd.read_csv('../data/raw/hydrography/gsl_nwm_gage.csv')
well_gdf = gpd.read_file('../data/raw/hydrography/well_shp.shp')
stream_gdf = gpd.read_file('../data/raw/hydrography/gslb_stream.shp')
lake_gdf = gpd.read_file('../data/raw/hydrography/lake.shp')

# find downstream gage

In [11]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

# Convert gage dataframe to GeoDataFrame (to match with subbasin_gdf)
# Create point geometries from longitude/latitude coordinates
gage_gdf = gpd.GeoDataFrame(
    gage_df,
    geometry=gpd.points_from_xy(gage_df['longitude'], gage_df['latitude']),
    crs=subbasin_gdf.crs
)

# Spatial join to find which subbasin each gage falls within
# Only keep gages that are within a subbasin (inner join)
matched_gages = gpd.sjoin(
    gage_gdf[['id', 'name', 'geometry']],
    subbasin_gdf[['linkno', 'geometry']],
    how='inner',
    predicate='within'
).rename(columns={'linkno': 'catchment_id'})

# Keep only relevant columns
matched_gages = matched_gages[['id', 'name', 'geometry', 'catchment_id']]



In [12]:
# Create a directed graph representing the river network
import networkx as nx
G = nx.DiGraph()

# Add edges to graph based on stream connectivity
# DSLINKNO represents downstream link number
for _, row in stream_gdf.iterrows():
    if pd.notna(row['DSLINKNO']) and row['DSLINKNO'] > 0:
        G.add_edge(int(row['LINKNO']), int(row['DSLINKNO']))

# Create dictionary mapping gage IDs to their catchment IDs
gage_links = dict(zip(matched_gages['id'], matched_gages['catchment_id']))
terminal_ids = []

# Find terminal gages (those that don't flow to any other gage)
for g1_id, g1_link in gage_links.items():
    is_terminal = True
    for g2_id, g2_link in gage_links.items():
        if g1_id != g2_id:
            try:
                # If there's a path from g1 to g2, then g1 is not terminal
                if nx.has_path(G, g1_link, g2_link):
                    is_terminal = False
                    break
            except:
                continue
    if is_terminal:
        terminal_ids.append(g1_id)

In [13]:
# Manual adjustments to terminal gages list based on domain knowledge
gages_to_remove = [10171000, 10167000]
gages_to_add = [10163000, 10153100, 10152000]

In [14]:
# Remove specified gages
terminal_ids = [gage_id for gage_id in terminal_ids if gage_id not in gages_to_remove]

# Add new gages if they exist in matched_gages
for gage_id in gages_to_add:
    if gage_id in matched_gages['id'].values and gage_id not in terminal_ids:
        terminal_ids.append(gage_id)

# Create dataframe of terminal gages
terminal_gages = matched_gages[matched_gages['id'].isin(terminal_ids)].copy()

# Find all upstream catchments for each terminal gage
records = []
for _, gage in terminal_gages.iterrows():
    upstream_ids = set()
    # Check each node in the graph
    for node in G.nodes:
        # If there's a path from node to terminal gage's catchment,
        # then this node is upstream
        if nx.has_path(G, node, gage['catchment_id']):
            upstream_ids.add(node)
    # Include the terminal gage's own catchment
    upstream_ids.add(gage['catchment_id'])

    # Create records for each upstream catchment
    for up_id in upstream_ids:
        records.append({
            'Gage_ID': gage['id'],
            'Gage_Name': gage['name'],
            'Terminal_Catchment_ID': gage['catchment_id'],
            'Upstream_Catchment_ID': up_id
        })

# Create and save the final dataframe
df_upstream = pd.DataFrame(records)
df_upstream.to_csv("../data/processed/terminal_gage_upstream_catchments.csv", index=False)

In [16]:
print(f'Number of unique gages in df_upstream: {df_upstream["Gage_ID"].nunique()}')
print('\nTerminal gage names:')
for _, row in df_upstream[['Gage_ID', 'Gage_Name']].drop_duplicates().iterrows():
    print(f'{row["Gage_ID"]}: {row["Gage_Name"]}')


Number of unique gages in df_upstream: 12

Terminal gage names:
10126000: BEAR RIVER NEAR CORINNE - UT
10141000: WEBER RIVER NEAR PLAIN CITY - UT
10142000: FARMINGTON CR ABV DIV NR FARMINGTON - UTAH
10143500: CENTERVILLE CREEK ABV. DIV NEAR CENTERVILLE - UT
10152000: SPANISH FORK NEAR LAKE SHORE - UTAH
10153100: HOBBLE CREEK AT 1650 WEST AT SPRINGVILLE - UTAH
10163000: PROVO RIVER AT PROVO - UT
10168000: LITTLE COTTONWOOD CREEK @ JORDAN RIVER NR SLC
10168500: BIG COTTONWOOD CR NR SALT LAKE CITY UTAH
10172700: VERNON CREEK NEAR VERNON - UT
10172860: WARM CREEK NEAR GANDY - UT
10172952: DUNN CREEK NEAR PARK VALLEY - UT
